In [135]:
import pandas as pd
import numpy as np
import scipy.stats
from collections import Counter
import scipy
from scipy.stats import beta
from scipy.stats import t

## Question 1: Like in class, we want to make an assessment, for each campaign, of its true click-through-rate (abbreviated as "c") and its true average post-click transaction profit volume (abbreviated as "v"). You can think of "v" as the average profit from a click or conversion. We want to make an assessment, for each campaign,  also of its true expected volume per exposure or impression (abbreviated as "EVI"). The true EVI of a campaign is its "c" multiplied by its "v".  For each campaign, compute the following:


### 1. The bayesian posterior probability that the campaign's true click-through-rate is the highest across all campaigns. Assume the prior is the uniform distribution. To generate draws from the beta distribution, in R use the rbeta() function and in Python use scipy.stats.beta.rvs(a=alpha value, b=beta value, size=number of draws needed). Here alpha value and beta value are, respectively, what we referred to in the sessions as the success parameter and the failure parameter.

In [136]:
volumes = pd.read_excel('volumes.dataset.2.xlsx')
clicks = pd.read_excel("clicks.dataset.2.xlsx").T
clicks.columns = clicks.iloc[0]
clicks = clicks[1:]

clicks['success'] = clicks['clicks']
clicks['failure'] = clicks['exposures'] - clicks['clicks']
clicks['beta_success'] = clicks['clicks'] + 1
clicks['beta_failure'] = clicks['failure'] + 1

In [142]:
draw_1 = [0 for _ in range(5)]
for i in range(len(draw_1)):
    draw_1[i] += scipy.stats.beta.rvs(a=clicks.iloc[i][4],
                                      b=clicks.iloc[i][5], 
                                      size=100000)
    draw_1[i] = draw_1[i].tolist()



combined_draw_1 = np.stack(draw_1, axis=1)
max_num_1 = np.amax(combined_draw_1, axis=1)
combined_draw_1 = np.stack([*draw_1, max_num_1], axis=1)

for i in range(len(combined_draw_1)):
    for j in range(5):
        if combined_draw_1[i][j] == combined_draw_1[i][5]:
            combined_draw_1[i][j] = 1
        else:
            combined_draw_1[i][j] = 0
            
            
ave_campaign_1 = combined_draw_1.mean(axis=0)
ave_campaign_1 = ave_campaign_1[:5]

max_c = np.max(ave_campaign_1)
index_max_c = np.where(ave_campaign_1 == max_c)[0][0]

print("The bayesian posterior probability that the campaign's true click-through-rate is the highest across all campaigns is", max_c, "for campaign", index_max_c + 1, ".")
      
      
      

The bayesian posterior probability that the campaign's true click-through-rate is the highest across all campaigns is 0.46737 for campaign 1 .


### 2. The bayesian posterior probability that the campaign's true average post-click volume per click is the highest across all campaigns. To generate draws from the t distribution in R use the rt() function. In Python, first generate draws from the standardized t distribution using scipy.stats.t.rvs(df= the appropriate degrees of freedom, size=number of draws needed). Once you have the draws in either R or Python you have to multiply by the standard error and then add the sample mean as shown in the example Excel computation to obtain the draws from the target posterior probability density.

In [141]:
grouped_ads = volumes.groupby('ad')
mean_volume = grouped_ads.mean()
var_volume = grouped_ads.var()
cnt_volume = grouped_ads.count()
stats = pd.DataFrame(columns=['ad', 'mean', 'se', 'df'])
stats['ad'] = [1, 2, 3, 4, 5]
stats['mean'] = mean_volume['volume'].tolist()
stats['se'] = (np.sqrt(var_volume['volume']) / np.sqrt(cnt_volume['volume'])).tolist()
stats['df'] = (cnt_volume['volume'] + 1).tolist() 

In [145]:
draw_2 = [0 for _ in range(5)]
for i in range(len(stats)):
    draw_2[i] += scipy.stats.t.rvs(df=stats.iloc[i,3], size=100000) * stats.iloc[i,2] + stats.iloc[i,1]
    draw_2[i] = draw_2[i].tolist()



combined_draw_2 = np.stack(draw_2, axis=1)
max_num_2 = np.amax(combined_draw_2, axis=1)
combined_draw_2 = np.stack([*draw_2, max_num_2], axis=1)

for i in range(len(combined_draw_2)):
    for j in range(5):
        if combined_draw_2[i][j] == combined_draw_2[i][5]:
            combined_draw_2[i][j] = 1
        else:
            combined_draw_2[i][j] = 0

ave_campaign_2 = combined_draw_2.mean(axis=0)
ave_campaign_2 = ave_campaign_2[:5]

max_v = np.max(ave_campaign_2)
index_max_v = np.where(ave_campaign_2 == max_v)[0][0]

print("The bayesian posterior probability that the campaign's true average post-click volume per click is the highest across all campaigns is", max_v, "for campaign", index_max_v + 1, ".")
      
      
     

The bayesian posterior probability that the campaign's true average post-click volume per click is the highest across all campaigns is 0.99999 for campaign 5 .


### 3. The bayesian posterior probability that the campaign's true expected volume per exposure (impression) is the highest.

In [146]:
draw_3 = [0 for _ in range(5)]
for i in range(len(draw_3)):
    draw_3[i] += np.array(draw_1)[i] * np.array(draw_2)[i]
    draw_3[i] = draw_3[i].tolist()



combined_draw_3 = np.stack(draw_3, axis=1)
max_num_3 = np.amax(combined_draw_3, axis=1)
combined_draw_3 = np.stack([*draw_3, max_num_3], axis=1)

for i in range(len(combined_draw_3)):
    for j in range(5):
        if combined_draw_3[i][j] == combined_draw_3[i][5]:
            combined_draw_3[i][j] = 1
        else:
            combined_draw_3[i][j] = 0

ave_campaign_3 = combined_draw_3.mean(axis=0)
ave_campaign_3 = ave_campaign_3[:5]

max_3 = np.max(ave_campaign_3)
index_max_3 = np.where(ave_campaign_3 == max_3)[0][0]

print("The bayesian posterior probability that the campaign's true expected volume per exposure (impression) is the highest is", max_3, "for campaign", index_max_3 + 1, ".")
      
      
     

The bayesian posterior probability that the campaign's true expected volume per exposure (impression) is the highest is 0.34182 for campaign 5 .


## Question 2 (Optional for Extra Credit): Campaigns with lower CTR often tend to have higher post-click volume. Explain why in 2-3 sentences. This question is on marketing and consumer psychology rather that statistics. Note: the comparison across campaigns is within the same category of campaigns (example, within banner ads of the same size, or within video ads of the same duration).

Campaigns with lower CTR draw in a wider spectrum of viewers, capturing the interest of individuals who may not have
clicked on the ad initially but remain intrigued by the product or service. Consequently, these campaigns generate increased
conversions from users influenced by the ad despite not clicking on it immediately. Moreover, such campaigns may leverage
more compelling content or precise targeting tactics, fostering greater post-click engagement despite the lower number of
initial clicks.

## Question 3 (Optional for Extra Credit): Consider the ordering of probabilities of campaigns in terms of the probability of having the highest CTR*v. Next consider the ordering of the campaigns in terms of ObservedCTR*MeanVolume. Will these two orderings always be the same? Assume that the number of draws is gigantic so that the Monte Carlo simulation error is negligible. Hint: This is directly related to the following question. Suppose x and y are two independent random variables with respective densities f(x) and f(y). If E(x) is greater than E(y), then is it always true that P(x>y)  is greater than P(y>x), or equivalently that P(x>y) is greater than half? If you say  this is always true, then give the rationale or proof. If you say it is not always true then give an example where it is not true and also the rationale for the failure.

These two orderings will not always be the same.

Counterexample: X ~ N(2, sigma_1) , Y ~ N(1, sigma_1) It is possible that E(X)>E(Y) but P(X>Y) is not greater than P(Y>X) or
greater than half.

Rationale: Although X has a higher expected value, there is still a significant probability mass of Y exceeding X due to
variability in the distributions.

## Question 4 Using generative AI tools

## (a)
1. unpack a list in python

## (b)
ChatGpt

## (c)

In [ ]:
# 1
list1 = [1, 2, 3]
list2 = [4, 5, 6]

combined_list = [*list1, *list2]
print(combined_list) # Output: [1, 2, 3, 4, 5, 6]